<a href="https://colab.research.google.com/github/Dan5049/MPC-MLF/blob/main/Final%20project/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final project
## MPA-MLF
**Daniel Havranek, Tomas Kricka**


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.datasets import cifar10
from keras.utils import to_categorical, plot_model, model_to_dot
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from keras.backend import clear_session